In [32]:
import pandas as pd
from datetime import datetime

In [42]:
import pandas as pd
from datetime import datetime

# Load the data
# Assuming sp500_constituents.csv has columns: 'Firm', 'EntryDate', 'ExitDate'
df = pd.read_csv("../Code_4_SECfilings/sp500_constituents.csv")

# Convert date columns to datetime format
df['start'] = pd.to_datetime(df['start'], errors='coerce')
df['ending'] = pd.to_datetime(df['ending'], errors='coerce')
df['nameendt'] = pd.to_datetime(df['nameendt'], errors='coerce')  # Ensure nameendt is datetime
# Define the range of years we are interested in
years = range(2006, 2025)

# Dictionary to hold the yearly snapshots
sp500_by_year = {}
permno_to_ticker = {}
for year in years:
    # Define the start and end of each year
    start_of_year = datetime(year, 1, 1) # 2006-01-01
    end_of_year = datetime(year, 12, 31) # 2006-12-31
    
    # Filter firms active during the year, ensuring they only appear once per year by permno
    active_firms = df[
        (df['start'] <= end_of_year) & 
        ((df['ending'].isna()) | (df['ending'] >= start_of_year))
    ]
    # Get the last entry(nameendt) for each permno, and remove permno duplicate except the last entry of active_firms
    active_firms = active_firms.sort_values(by=['permno', 'nameendt']).groupby('permno').last().reset_index()
    # Convert the resulting DataFrame of firms to a list of unique permnos
    permno_to_ticker = dict(zip(active_firms['permno'], active_firms['ticker']))

    # Store the list of active firms for the year
    sp500_by_year[year] = permno_to_ticker
    

# print(sp500_by_year)
# # Display the results for each year
# for year, firms in sp500_by_year.items():
#     print(f"Year {year}: {len(firms)} firms")
#     # Uncomment below to see the list of firms per year
#     print(firms.values())


In [43]:
path = '../Code_4_SECfilings/update_only2025.csv'
path = '/Users/apple/PROJECT/Code_4_SECfilings/sp500_total_constituents.csv'
df = pd.read_csv(path)
df = df.rename(columns={"Symbol": "Ticker"})
reversed_dict = {}
for year, firms in sp500_by_year.items():
    reversed_dict[year] = {ticker: pernmo for pernmo, ticker in sp500_by_year[year].items()}
    for permno, ticker in sp500_by_year[year].items():
        if ticker in df["Ticker"].tolist():
            cik = df[df["Ticker"] == ticker]["CIK"].values[0]
            reversed_dict[year][ticker] = cik


In [44]:
sp500_by_year = reversed_dict.copy()


In [ ]:
file_path = "/Users/apple/PROJECT/hons_project/data/SP500/analysis_reports/intermediate/batch_10.parquet"
# Load the intermediate Parquet file to be filtered 
sp500_dtm = pd.read_parquet(file_path)
sp500_dtm["Date"] = pd.to_datetime(sp500_dtm["Date"], errors="coerce") 
sp500_dtm['Year'] = sp500_dtm["Date"].dt.year


# Add fake row
# Define the placeholder row dynamically
placeholder_row = {col: 0.0 for col in sp500_dtm.columns}
placeholder_row['Year'] = 2024

# Add the placeholder row to the DataFrame
sp500_dtm.loc[len(sp500_dtm)] = placeholder_row
print(sp500_dtm[sp500_dtm["Year"] == 2024])
# Create a set of valid CIK-Year pairs
valid_pairs = set()
for year, firms in sp500_by_year.items():
    for cik in firms.values():
        cik = str(cik).zfill(10)
        valid_pairs.add((year, cik))
valid_years = [year for year, _ in valid_pairs]



     Date _cik  _vol  _ret  _vol+1  _ret+1    p  water  corporation  span  \
6825  0.0  0.0   0.0   0.0     0.0     0.0  0.0    0.0          0.0   0.0   

      ...  bennet  insurrection  bari  smother  cachet  healthily  neve  \
6825  ...     0.0           0.0   0.0      0.0     0.0        0.0   0.0   

      ideology  sped  Year  
6825       0.0   0.0  2024  

[1 rows x 14214 columns]
valid_pairs {(2006, '0000048267'), (2019, '0000922864'), (2006, '0001841968'), (2019, '0001633917'), (2022, '0000063754'), (2021, '0000105770'), (2021, '0000277948'), (2019, '0001596532'), (2018, '0000106040'), (2015, '0000745732'), (2019, '0001442145'), (2017, '0001137774'), (2014, '0001035002'), (2006, '0000022779'), (2007, '0001571996'), (2023, '0000896159'), (2007, '0000049826'), (2016, '0000050863'), (2019, '0001133421'), (2018, '0000074208'), (2011, '0000816761'), (2011, '0001002910'), (2022, '0000813672'), (2015, '0000015797'), (2011, '0000036104'), (2011, '0000076639'), (2022, '0001781335'), (20

In [78]:
# path = "/Users/apple/PROJECT/hons_project/data/SP500/analysis_reports/intermediate/batch_10.parquet"
# batch_number = 10
# df = pd.read_parquet(path)
# df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
# df.shape

(6825, 14213)

In [79]:
# df['Year'] = df["Date"].dt.year

# vaild_pairs = set()
# for year, firms in sp500_by_year.items():
#     for cik in firms.values():
#         cik = str(cik).zfill(10)
#         vaild_pairs.add((year, cik))
# df_filtered = df[df.apply(lambda row: (row["Year"], row["_cik"]) in vaild_pairs, axis=1)]
# df_filtered = df_filtered.drop(columns=["Year"])
# df_filtered.shape

(5897, 14213)

In [80]:
df_filtered.to_parquet(f"/Users/apple/PROJECT/hons_project/data/SP500/analysis_reports/filtered/batch_filtered_{batch_number}.parquet")